# Генетическая стратификация

Этот ноутбук показывает, как использовать библиотеку **GenRest**.
Мы создадим синтетические данные с категориальными и числовыми
признаками, преобразуем числовой признак в категории, а затем
запустим `GeneticStratifier`.

In [1]:
import pandas as pd
import numpy as np
from genrest import GeneticStratifier, bin_numeric

rng = np.random.default_rng(0)
colors = rng.choice(['red', 'blue', 'green'], size=200)
shapes = rng.choice(['circle', 'square', 'triangle'], size=200)
age = rng.normal(30, 10, size=200)
y = ((colors == 'red').astype(int) + (shapes == 'circle').astype(int) + age / 10 + rng.normal(0, 1, 200))
data = pd.DataFrame({'color': colors, 'shape': shapes, 'age': age, 'y': y})
data.head()

  color     shape        age         y0   red    circle  41.600000  4.5000001  blue  triangle  27.300000  2.2000002 green    square  25.000000  1.0000003   red    square  38.700000  3.8000004  blue    circle  30.500000  3.000000

Преобразуем числовой столбец `age` в категории с помощью функции `bin_numeric` и посмотрим на результат.

In [2]:
bin_numeric(data, 'age', bins=3)
data.head()

  color     shape           age         y0   red    circle  (33.7, inf]  4.5000001  blue  triangle  (28.5, 33.7]  2.2000002 green    square   (-inf, 28.5]  1.0000003   red    square  (33.7, inf]  3.8000004  blue    circle  (28.5, 33.7]  3.000000

Теперь запустим генетический алгоритм. В качестве параметра `n_groups` укажем 2, чтобы получить 2^3=8 страт.

In [3]:
stratifier = GeneticStratifier(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    population_size=10,
    generations=5,
    n_groups=2,
    random_state=0,
)
best = stratifier.fit(data)
best, stratifier.best_score_

(Stratification(boundaries={'color': {'red': 1, 'blue': 0, 'green': 0}, 'shape': {'circle': 1, 'square': 0, 'triangle': 0}, 'age': {'(-inf, 28.5]': 0, '(28.5, 33.7]': 1, '(33.7, inf]': 0}}), 0.21)